In [158]:
import numpy as np
import nway
import os
import matplotlib.pyplot as plt
from dandi import dandiapi
import matplotlib as mpl
from pynwb import NWBHDF5IO
from nway.meta_registration import MetaRegistration
import scipy
from PIL import Image
import json

In [4]:
dandiset_id = "000336"
dandi_filepath = "sub-1168633075/sub-1168633075_ses-1193555033-1193675745-_ophys.nwb"

authenticate = True
dandi_api_key = os.environ["dandi_api_key"]

In [5]:
if authenticate:
    client = dandiapi.DandiAPIClient(token=dandi_api_key)
else:
    client = dandiapi.DandiAPIClient()
my_dandiset = client.get_dandiset(dandiset_id)

filename = dandi_filepath.split("/")[-1]
filepath = f"./{filename}"
file = my_dandiset.get_asset_by_path(dandi_filepath)

A newer version (0.51.0) of dandi/dandi-cli is available. You are using 0.46.3


In [6]:
# # this may take awhile, especially if the file to download is large
# file.download(filepath)

# print(f"Downloaded file to {filepath}")

In [7]:
data_dir = "../../../data/dendrite_coupling"
io = NWBHDF5IO(f"{data_dir}/sub-1168633075_ses-1193555033-1193675745-_ophys.nwb", mode="r", load_namespaces=True)
nwb1 = io.read()

io = NWBHDF5IO(f"{data_dir}/sub-1168633075_ses-1193555033-1193675748-_ophys.nwb", mode="r", load_namespaces=True)
nwb2 = io.read()

C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-common' version 1.5.0 because version 1.5.1 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'core' version 2.3.0 because version 2.5.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."
C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\hdmf\spec\namespace.py:531: UserWarning: Ignoring cached namespace 'hdmf-experimental' version 0.1.0 because version 0.2.0 is already loaded.
  warn("Ignoring cached namespace '%s' version %s because version %s is already loaded."


In [8]:
nwbs = [nwb1, nwb2]

In [21]:
nwb1

root pynwb.file.NWBFile at 0x2838590993072
Fields:
  acquisition: {
    EyeTracking <class 'abc.EllipseEyeTracking'>,
    v_in <class 'pynwb.base.TimeSeries'>,
    v_sig <class 'pynwb.base.TimeSeries'>
  }
  devices: {
    MESO.2 <class 'pynwb.device.Device'>
  }
  experiment_description: ophys session
  file_create_date: [datetime.datetime(2022, 12, 25, 2, 26, 18, 277733, tzinfo=tzutc())]
  identifier: 1193675745
  imaging_planes: {
    imaging_plane_1 <class 'pynwb.ophys.ImagingPlane'>
  }
  institution: Allen Institute for Brain Science
  intervals: {
    fixed_gabors_presentations <class 'pynwb.epoch.TimeIntervals'>,
    gratings_presentations <class 'pynwb.epoch.TimeIntervals'>,
    movie_flower_fwd_presentations <class 'pynwb.epoch.TimeIntervals'>,
    movie_touch_of_evil_fwd_presentations <class 'pynwb.epoch.TimeIntervals'>,
    movie_worms_fwd_presentations <class 'pynwb.epoch.TimeIntervals'>,
    rotate_gabors_presentations <class 'pynwb.epoch.TimeIntervals'>,
    spontaneous_

### Convert to Nwaymatching Input File

In [ ]:
# for each nwb file
    # save projection image
    # get/generate id?
    # for each roi
        # get roi attributes
        # get roi mask as boolean

In [37]:
nwb.processing["ophys"]["image_segmentation"]["cell_specimen_table"][:10]

,cell_specimen_id,height,mask_image_plane,max_correction_down,max_correction_left,max_correction_right,max_correction_up,valid_roi,width,x,y,image_mask
id,,,,,,,,,,,,
1197269239,-1,15,1,3.0,8.0,7.0,3.0,True,19,243,442,"[[False, False, False, False, False, False, Fa..."
1197269266,-1,14,1,3.0,8.0,7.0,3.0,True,18,38,279,"[[False, False, False, False, False, False, Fa..."
1197269270,-1,13,0,3.0,8.0,7.0,3.0,True,21,332,431,"[[False, False, False, False, False, False, Fa..."
1197269293,-1,21,0,3.0,8.0,7.0,3.0,True,19,126,475,"[[False, False, False, False, False, False, Fa..."
1197269296,-1,18,0,3.0,8.0,7.0,3.0,True,23,142,342,"[[False, False, False, False, False, False, Fa..."
1197269337,-1,22,0,3.0,8.0,7.0,3.0,True,22,245,382,"[[False, False, False, False, False, False, Fa..."
1197269361,-1,12,0,3.0,8.0,7.0,3.0,True,17,371,259,"[[False, False, False, False, False, False, Fa..."
1197269363,-1,13,0,3.0,8.0,7.0,3.0,True,21,164,238,"[[False, False, False, False, False, False, Fa..."
1197269374,-1,17,0,3.0,8.0,7.0,3.0,True,12,122,270,"[[False, False, False, False, False, False, Fa..."


In [151]:
def make_roi_entries(roi_table):
    roi_entries = []
    for roi_idx in range(len(roi_table)):
        x = int(roi_table["x"][roi_idx])
        y = int(roi_table["y"][roi_idx])
        height = int(roi_table["height"][roi_idx])
        width = int(roi_table["width"][roi_idx])
        
        mask_subimage = roi_table["image_mask"][roi_idx][y:y+height,x:x+width]
        
        roi_entry = {
            "id": int(roi_table["id"][roi_idx]),
            "valid": bool(roi_table["valid_roi"][roi_idx]),
            "x": x,
            "y": y,
            "z": 1,
            "width": width,
            "height": height,
            # convert 2D array to 2D list of bools
            "mask_matrix": [[bool(elem) for elem in row] for row in mask_subimage]
        }
        roi_entries.append(roi_entry)
    return roi_entries


In [173]:
def nwb_to_experiment_json(nwb):
    # save image here
    image = np.array(nwb.processing["ophys"]["images"]["average_image"])
    print(image.shape)
    print(image)
    print(type(image[0][0]))
    image = Image.fromarray(image)
    image.save(f"./{nwb.identifier}_image.png")

    roi_table = nwb.processing["ophys"]["image_segmentation"]["cell_specimen_table"]
    
    experiment = {
        "id": nwb.identifier, 
        "ophys_average_intensity_projection_image": f"./{nwb.identifier}_image.png",
        "cell_rois": make_roi_entries(roi_table)
    }

    return experiment

In [174]:
input_exps = {
    "experiment_containers": {
        "ophys_experiments": []
    }
}

for nwb in nwbs:
    experiment = nwb_to_experiment_json(nwb)
    input_exps["experiment_containers"]["ophys_experiments"].append(experiment)

with open("input.json", "w") as f:
    json.dump(input_exps, f, indent=2)

(512, 512)
[[0.04313726 0.04313726 0.04313726 ... 0.04705882 0.04705882 0.03921569]
 [0.04313726 0.04313726 0.04313726 ... 0.04313726 0.04313726 0.03921569]
 [0.04313726 0.04313726 0.04313726 ... 0.04313726 0.04313726 0.03921569]
 ...
 [0.04705882 0.04705882 0.04705882 ... 0.04705882 0.04705882 0.04313726]
 [0.04705882 0.04705882 0.04705882 ... 0.04705882 0.04705882 0.04313726]
 [0.03921569 0.03921569 0.03921569 ... 0.03921569 0.03921569 0.03529412]]
<class 'numpy.float32'>


OSError: cannot write mode F as PNG

In [154]:
!python -m nway.nway_matching --input_json input.json --output_json test_output/output.json --output_dir test_output

INFO:NwayMatching:NWAY_COMMIT_SHA None
INFO:NwayMatching:Nway matching version 0.6.0
Traceback (most recent call last):
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 197, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\runpy.py", line 87, in _run_code
    exec(code, run_globals)
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\nway\nway_matching.py", line 502, in <module>
    nmod.run()
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\nway\nway_matching.py", line 435, in run
    self.make_masks_from_dicts()
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\nway\nway_matching.py", line 181, in make_masks_from_dicts
    utils.create_nice_mask(
  File "C:\Users\carter.peene\AppData\Local\Programs\Python\Python39\lib\site-packages\nway\utils.p